# Graph Theoretical Analysis

In [1]:
import os
import pandas as pd
import networkx as nx
import numpy as np
import seaborn as sns
import SigProc
from analysis import graph_analysis_utils as gau
from analysis import analysis_utils as au
from scipy import stats
from IPython.core.interactiveshell import InteractiveShell

In [2]:
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline

In [3]:
mouse_directory = os.path.join(os.path.expanduser("~"), "Hen_Lab/Mice/EPM")

if not os.path.exists(mouse_directory):
    print("The mouse directory does not exist", file=sys.stderr)

animal_data_files = dict()

for dir_name, subdir_list, file_list in os.walk(mouse_directory):
    if subdir_list:
        mice_directories = subdir_list

    for file_name in file_list:
        for mouse_name in mice_directories:
            if file_name.endswith(".csv") and (mouse_name in file_name and "behavior" in file_name):
                if mouse_name in animal_data_files:
                    animal_data_files[mouse_name].update({"behavior": os.path.join(dir_name, file_name)})
                else:
                    animal_data_files[mouse_name] = {"behavior": os.path.join(dir_name, file_name)}
            elif file_name.endswith(".csv") and (mouse_name in file_name and "Raw" in file_name):
                if mouse_name in animal_data_files:
                    animal_data_files[mouse_name].update({"Raw": os.path.join(dir_name, file_name)})
                else:
                    animal_data_files[mouse_name] = {"Raw": os.path.join(dir_name, file_name)}

## Store all neuron activity and corresponding behavior dataframes, for each animal, in a dictionary

In [4]:
animal_dataframes = dict()

for animal_name in animal_data_files:
    print(animal_name)
    
    data = pd.read_csv(animal_data_files[animal_name]["Raw"], header=None)
    
    if animal_name == "drd73" or animal_name == "drd77":
        data = au.downsample_dataframe(data,2)
    _, _, cell_transients_dataframe = SigProc.detect_ca_transients_mossy(data, 2, 0.5, 0.2, 10)
    
    # Rename all the columns from "neuron_x" --> "x". This makes the graphs neater by making sure the neuron names fit into the nodes
    cell_transients_dataframe.columns = [i for i in range(1, len(cell_transients_dataframe.columns)+1)]
    
    behavior_column_names = ['Trial_time', 'Recording_time', 'X_center', 'Y_center', 'Area', 'Areachange', 
                         'Elongation', 'Distance_moved', 'Velocity', 'Arena_centerpoint',
                         'Open1_centerpoint', 'Open2_centerpoint',
                         'Closed1_centerpoint', 'Closed2_centerpoint',
                         'OpenArms_centerpoint', 'ClosedArms_centerpoint', 'Result_1']

    behavior_df = pd.read_csv(animal_data_files[animal_name]["behavior"], header=None)
    behavior_df.columns = behavior_column_names
    
    # Only keep every 3rd row of the original behavior dataframe in order to downsample 30 fps --> 10 fps
    behavior_df = au.downsample_dataframe(behavior_df, 3)

    # Define cutoff for what constitutes as a running frame
    VELOCITY_CUTOFF = 4;

    # Adds "Running_frames" column to the end of the behavior dataframe 
    behavior_df["Running_frames"] = np.where(behavior_df["Velocity"] > VELOCITY_CUTOFF, 1, 0)
    
    neuron_concated_behavior = cell_transients_dataframe.join(behavior_df, how="left")
    
    if not animal_name in animal_dataframes:
        animal_dataframes[animal_name] = {"cell transients": cell_transients_dataframe, "behavior": behavior_df, "neuron and beh": neuron_concated_behavior}

drd87
drd73
drd77
drd46


## Compute all network measures for each mouse, and store them in a dictionary

In [25]:
import plotly
import plotly.graph_objs as go

def plot_network_measures(y, y2, *animal_names, **kwargs):
    trace1 = go.Bar(
        x=animal_names,
        y=y,
        text=y,
        textposition = "auto",
        name="Open Arms",
        marker=dict(color="rgb(255, 0, 0)"),
        opacity=0.6
    )
    trace2 = go.Bar(
        x=animal_names,
        y=y2,
        text=y2,
        textposition = "auto",
        name="Closed Arms",
        marker=dict(color="rgb(58,200,225)"),
        opacity=0.6
    )
    data = [trace1, trace2]
    layout = go.Layout(
        title = kwargs["title"],
    )
    fig = go.Figure(data=data, layout=layout)
    plotly.offline.iplot(fig)

In [26]:
def get_measures_for_plot(function):
    """Makeshift function that was implemented only for the prevention of repetition of code.
    
        This function simply runs through each animal's cell trasients events DataFrame and uses
        that DataFrame to execute a certain network measure function on it, e.g. degree centrality.
    """
    labels = list()
    open_arms_measures = list()
    closed_arms_measures = list()

    for animal in animal_dataframes:
        labels.append(animal)
        indices = animal_dataframes[animal]["neuron and beh"].loc[animal_dataframes[animal]["neuron and beh"]["OpenArms_centerpoint"] != 0].index
        open_arms_graph = gau.create_graph(animal_dataframes[animal]["cell transients"].iloc[indices])

        # Compute OpenArms Network measure
        open_arms_measures.append(function(open_arms_graph))

        indices = animal_dataframes[animal]["neuron and beh"].loc[animal_dataframes[animal]["neuron and beh"]["ClosedArms_centerpoint"] != 0].index
        closed_arms_graph = gau.create_graph(animal_dataframes[animal]["cell transients"].iloc[indices])

        # Compute ClosedArms Network measure
        closed_arms_measures.append(function(closed_arms_graph))
        
    return labels, open_arms_measures, closed_arms_measures

In [28]:
labels, open_arms_measures, closed_arms_measures = get_measures_for_plot(gau.compute_mean_degree_centrality)
plot_network_measures(open_arms_measures, closed_arms_measures, *labels, title="Mean Degree Centrality")

labels, open_arms_measures, closed_arms_measures = get_measures_for_plot(gau.compute_connection_density)
plot_network_measures(open_arms_measures, closed_arms_measures, *labels, title="Connection Density")

labels, open_arms_measures, closed_arms_measures = get_measures_for_plot(nx.average_clustering)
plot_network_measures(open_arms_measures, closed_arms_measures, *labels, title="Clustering Coefficient")

labels, open_arms_measures, closed_arms_measures = get_measures_for_plot(gau.compute_mean_clique_size)
plot_network_measures(open_arms_measures, closed_arms_measures, *labels, title="Mean Clique Size")

labels, open_arms_measures, closed_arms_measures = get_measures_for_plot(gau.get_max_clique_size)
plot_network_measures(open_arms_measures, closed_arms_measures, *labels, title="Max Clique Size")

labels, open_arms_measures, closed_arms_measures = get_measures_for_plot(nx.global_efficiency)
plot_network_measures(open_arms_measures, closed_arms_measures, *labels, title="Global Efficiency")

labels, open_arms_measures, closed_arms_measures = get_measures_for_plot(nx.local_efficiency)
plot_network_measures(open_arms_measures, closed_arms_measures, *labels, title="Local Efficiency")

labels, open_arms_measures, closed_arms_measures = get_measures_for_plot(gau.compute_mean_betweenness_centrality)
plot_network_measures(open_arms_measures, closed_arms_measures, *labels, title="Mean Betweenness Centrality")

labels, open_arms_measures, closed_arms_measures = get_measures_for_plot(gau.compute_mean_load_centrality)
plot_network_measures(open_arms_measures, closed_arms_measures, *labels, title="Mean Load Centrality")